In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
# missing libs - metrics,

In [3]:
!pip install catalyst
!pip install albumentations

In [39]:


from pathlib import Path
from datetime import datetime
import numpy as np

import torch
import torch.nn as nn
from torch.utils import data

import catalyst
from catalyst import dl
from catalyst.utils import set_global_seed
from catalyst import metrics
from torchvision import transforms, models

In [40]:
set_global_seed(42)

In [41]:


from catalyst.utils import (
    create_dataset, create_dataframe, get_dataset_labeling, map_dataframe
)

dataset = create_dataset(dirs=f"Imagenette-comp/train/*", extension="*.jpg")
df = create_dataframe(dataset, columns=["class", "filepath"])

tag_to_label = get_dataset_labeling(df, "class")
class_names = [
    name for name, id_ in sorted(tag_to_label.items(), key=lambda x: x[1])
]

df_with_labels = map_dataframe(
    df, 
    tag_column="class", 
    class_column="label", 
    tag2class=tag_to_label, 
    verbose=False
)
df_with_labels.head()
df_with_labels.tail()



,class,filepath,label
8962,c1d70420,Imagenette-comp/train\c1d70420\e3934f8e.jpg,9
8963,c1d70420,Imagenette-comp/train\c1d70420\e3945fc8.jpg,9
8964,c1d70420,Imagenette-comp/train\c1d70420\e39504f0.jpg,9
8965,c1d70420,Imagenette-comp/train\c1d70420\e395bf1c.jpg,9
8966,c1d70420,Imagenette-comp/train\c1d70420\e396a4c2.jpg,9


In [42]:
from catalyst.utils import split_dataframe_train_test

train_data, valid_data = split_dataframe_train_test(
    df_with_labels, test_size=0.2, random_state=42
)
train_data, valid_data = (
    train_data.to_dict("records"),
    valid_data.to_dict("records"),
)

In [46]:

from catalyst.contrib.data.cv.reader import ImageReader
from catalyst import utils
from catalyst.data import ScalarReader, ReaderCompose

num_classes = len(tag_to_label)

open_fn = ReaderCompose(
    [
        ImageReader(
            input_key="filepath", output_key="features", rootpath="Imagenette-comp/train"
        ),
        ScalarReader(
            input_key="label",
            output_key="targets",
            default_value=-1,
            dtype=np.int64,
        ),
        ScalarReader(
            input_key="label",
            output_key="targets_one_hot",
            default_value=-1,
            dtype=np.int64,
            one_hot_classes=num_classes,
        ),
    ]
)



In [47]:


batch_size = 256
num_workers = 4

train_loader = utils.data.get_loader(
    train_data,
    open_fn=open_fn,
    batch_size=batch_size,
    num_workers=num_workers,
    shuffle=True,
    sampler=None,
    drop_last=True,
)

valid_loader = utils.data.get_loader(
    valid_data,
    open_fn=open_fn,
    batch_size=batch_size,
    num_workers=num_workers,
    shuffle=False, 
    sampler=None,
    drop_last=True,
)

loaders = {
    "train": train_loader,
    "valid": valid_loader
}



In [48]:


#model = MyResNet50()
model = models.resnet50(pretrained=True)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [36]:
runner = dl.SupervisedRunner()
# model training
runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    loaders=loaders,
    num_epochs=1,
    logdir="./logs",
    valid_loader="valid",
    valid_metric="loss",
    minimize_valid_metric=True,
    verbose=True,
# uncomment for extra metrics:
#     callbacks=[
#         dl.AccuracyCallback(input_key="logits", target_key="targets", num_classes=10),
#         dl.PrecisionRecallF1SupportCallback(
#             input_key="logits", target_key="targets", num_classes=10
#         ),
#         dl.AUCCallback(input_key="logits", target_key="targets"),
#         # catalyst[ml] required ``pip install catalyst[ml]``
#         dl.ConfusionMatrixCallback(
#             input_key="logits", target_key="targets", num_classes=num_classes
#         ),
#     ]
)

AttributeError: Can't pickle local object 'ListDataset.__init__.<locals>.<lambda>'

In [49]:
##with metrix - json
runner = ClassificationRunner(input_key="features", target_key="targets",output_key="logits",loss_key="loss")
runner.train(
    model=model,
    optimizer=optimizer,
    criterion=criterion,
    loaders=loaders,
    logdir=Path("logs") / datetime.now().strftime("%Y%m%d-%H%M%S"),
    num_epochs=10,
    verbose=True,
    load_best_on_end=True,
    callbacks={
        "optimizer": dl.OptimizerCallback(
            metric_key="loss", accumulation_steps=1, grad_clip_params=None,
        ),
        "criterion": dl.CriterionCallback(
            input_key="targets", target_key="logits", metric_key="loss",
        ),
        "accuracy": dl.AccuracyCallback(num_classes=10,input_key="logits", target_key="targets"),
    },
)



AttributeError: Can't pickle local object 'ListDataset.__init__.<locals>.<lambda>'

In [ ]:
'''

runner = dl.SupervisedRunner(
    input_key="features", output_key="logits", target_key="targets", loss_key="loss"
)

# model training
runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    loaders=loaders,
    num_epochs=1,
    callbacks=[
        dl.AccuracyCallback(input_key="logits", target_key="targets", topk_args=(1, 3, 5)),
        dl.PrecisionRecallF1SupportCallback(
            input_key="logits", target_key="targets", num_classes=10
        ),
    ],
    logdir="./logs",
    valid_loader="valid",
    valid_metric="loss",
    minimize_valid_metric=True,
    verbose=True,
    load_best_on_end=True,
)


'''

In [50]:
import pandas as pd
from PIL import Image
from tqdm.notebook import tqdm

submission = {"Id": [], "Category": []}
model.eval()

test_dataset = create_dataset(dirs=f"Imagenette-comp/test/", extension="*.jpg")
test_data = list({"filepath": filepath} for filepath in test_dataset["test"])

test_open_fn = ReaderCompose(
    [
        ImageReader(
            input_key="filepath", output_key="features", rootpath=""
        ),
        ScalarReader(
            input_key="filepath",
            output_key="filepath",
            default_value="",
            dtype=str,
        ),
    ]
)

test_loader = utils.get_loader(
    test_data,
    open_fn=test_open_fn,
    dict_transform=valid_data_transform,
    batch_size=batch_size,
    num_workers=num_workers,
    shuffle=False,
    sampler=None,
    drop_last=False,
)

for prediction in runner.predict_loader(loader=test_loader):
    prediction["labels"] = [class_names[c] for c in torch.max(prediction["log_probs"], axis=1)[1]]
    submission["Id"].extend(f.split("/")[4].split(".")[0] for f in prediction["filepath"])
    submission["Category"].extend(prediction["labels"])

PicklingError: Can't pickle <function <lambda> at 0x000002A8095C6168>: attribute lookup <lambda> on __main__ failed

In [51]:
pd.DataFrame(submission).to_csv("baseline.csv", index=False)